In [ ]:
import sagemaker
import boto3

sm_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sm_session.boto_region_name
account = sm_session.account_id()


container_name = "deepspeed-sm"
container_tag = "latest"
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, container_name, container_tag)

In [ ]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 763104351884.dkr.ecr.{region}.amazonaws.com
# loging to your private ECR
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account}.dkr.ecr.{region}.amazonaws.com

In [11]:
! ./build_and_push.sh $container_name $container_tag Dockerfile

Working in region us-east-1
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon    357MB
Step 1/37 : ARG REGION
Step 2/37 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04
1.6.0-gpu-py36-cu101-ubuntu16.04: Pulling from pytorch-training

fa4d2cf0: Pulling fs layer 
c939e38e: Pulling fs layer 
0ffd64f6: Pulling fs layer 
eed388b7: Pulling fs layer 
000175e2: Pulling fs layer 
a4e5b7f4: Pulling fs layer 
d843eff5: Pulling fs layer 
0a5bff61: Pulling fs layer 
c4b92f13: Pulling fs layer 
5fd10ed6: Pulling fs layer 
502379e5: Pulling fs layer 
1a683d2e: Pulling fs layer 
2af12cff: Pulling fs layer 
b6c7fa99: Pulling fs layer 
e2bf114a: Pulling

In [ ]:
! docker images ls

In [ ]:
! pip install 'sagemaker[local]' --upgrade

In [ ]:
from sagemaker.local import LocalSession

# Configure our local training session
sagemaker_local_session = LocalSession()
sagemaker_local_session.config = {'local': {'local_code': True}}

In [ ]:
! pygmentize training_container/cifar_ds_config.json

In [ ]:
est = sagemaker.estimator.Estimator(image,
                                    role=role,
                                    instance_count=1,
                                    instance_type='local_gpu',
                                    hyperparameters = {
                                        "epochs" : 2,
                                        "deepspeed-config-file":"cifar_ds_config.json",
                                        "train-script":"train_cifar10.py"
                                    },
                                    sagemaker_session=sagemaker_local_session
)

est.fit(inputs={"data":"file:///home/ec2-user/SageMaker/deepspeed-on-aws/data/"})

# Remote start

In [ ]:
est = sagemaker.estimator.Estimator(image,
                                    role=role,
                                    instance_count=2,
                                    instance_type='ml.p3.16xlarge',
                                    hyperparameters = {
                                        "epochs" : 20,
                                        "deepspeed-config-file":"cifar_ds_config.json",
                                        "train-script":"train_cifar10.py"                                        
                                    },
                                    sagemaker_session=sm_session
)

est.fit(inputs={"data":"s3://vadimd-cv-datasets/cifar-10-deepspeed/"})


2021-02-16 23:28:47 Starting - Starting the training job...
2021-02-16 23:29:11 Starting - Launching requested ML instancesProfilerReport-1613518127: InProgress
............
2021-02-16 23:31:13 Starting - Preparing the instances for training.........
2021-02-16 23:32:39 Downloading - Downloading input data
2021-02-16 23:32:39 Training - Downloading the training image....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-02-16 23:36:03,609 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-02-16 23:36:03,675 sagemaker-training-toolkit INFO     Failed to parse hyperparameter deepspeed-config-file value cifar_ds_config.json to Json.
Returning the value itself
2021-02-16 23:36:03,675 sagemaker-training-toolkit INFO     Failed to parse hyperparameter train-script value train_cifar10.py to Json.
Returning the value itself
2021-02-16 23:36:03,687 sagemaker_pytorch_co